!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-postgres -o yaml | yq

# Using Kubernetes Configuration Provider to load data from Secrets and Config Maps
   * https://strimzi.io/blog/2021/07/22/using-kubernetes-config-provider-to-load-data-from-secrets-and-config-maps/

In [12]:
!kubectl -n kafka-main-cluster get role connector-configuration-role -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"Role","metadata":{"annotations":{},"name":"connector-configuration-role","namespace":"kafka-main-cluster"},"rules":[{"apiGroups":[""],"resourceNames":["postgres-user-debezium"],"resources":["secrets"],"verbs":["get"]}]}
  creationTimestamp: "2024-01-31T21:11:15Z"
  name: connector-configuration-role
  namespace: kafka-main-cluster
  resourceVersion: "606513"
  uid: 11c59303-4a2d-4dfc-89bc-7dad23ed4627
rules:
  - apiGroups:
      - ""
    resourceNames:
      - postgres-user-debezium
    resources:
      - secrets
    verbs:
      - get


In [10]:
!kubectl -n kafka-main-cluster get rolebinding connector-configuration-role-binding -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"RoleBinding","metadata":{"annotations":{},"name":"connector-configuration-role-binding","namespace":"kafka-main-cluster"},"roleRef":{"apiGroup":"rbac.authorization.k8s.io","kind":"Role","name":"connector-configuration-role"},"subjects":[{"kind":"ServiceAccount","name":"my-connect-connect","namespace":"kafka-main-cluster"}]}
  creationTimestamp: "2024-01-31T21:11:15Z"
  name: connector-configuration-role-binding
  namespace: kafka-main-cluster
  resourceVersion: "606514"
  uid: 71900323-e3db-4879-bdf2-c772b891dcf4
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: connector-configuration-role
subjects:
  - kind: ServiceAccount
    name: my-connect-connect
    namespace: kafka-main-cluster


In [14]:
!kubectl -n kafka-main-cluster get secrets postgres-user-debezium -o yaml | yq

apiVersion: v1
data:
  password: ZGViZXppdW0K
  username: ZGViZXppdW0K
kind: Secret
metadata:
  annotations:
    replicator.v1.mittwald.de/replicated-at: "2024-01-31T16:10:45Z"
    replicator.v1.mittwald.de/replicated-from-version: "19115"
    replicator.v1.mittwald.de/replicated-keys: password,username
  creationTimestamp: "2024-01-31T16:10:45Z"
  name: postgres-user-debezium
  namespace: kafka-main-cluster
  resourceVersion: "19116"
  uid: 8434a02b-5d8e-42f8-bd19-6947356c3e03
type: Opaque
